In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as mp
import matplotlib.ticker as mtick

In [2]:
def getData(fileName, secondsPerMessage, sensor):
    # Daten importieren
    # Diese Daten liegen dem Anhang nicht bei, da es sich dabei um interne Daten des Donaubades handelt
    data = pd.read_csv('../../Nicht inkludierte Messwerte/' + fileName, skiprows=3, sep=',')
    
    print()
    print(sensor)
    
    if sensor != 'LoRaWAN':
        data = data[data['_measurement'] == sensor]
        data = data.reset_index(drop=True)
    
    data['time'] = pd.to_datetime(data['_time'], utc=True)

    # Abstand zwischen den Nachrichten
    data['time_delta'] = data['time'].diff().dt.total_seconds()

    data['received'] = True
        
        
    # Vom zeitliche Abstand jeder Nachricht wird der Soll-Abstand subtrahiert. Dadurch ergibt sich die Abweichung zum Soll.
    # Dann wird durch den Soll-Abstand dividiert und gerundet. So ergibt sich die Anzahl an Nachrichten, die jeweils zwischen zwei Nachrichten fehlen
    missingMessages = ((data['time_delta'][1:] - secondsPerMessage) / secondsPerMessage).round().astype(int)

    # Gibt nur die Anahl Nachrichten an, die gar nicht angekommen sind. Nachrichten dessen CRC fehlgeschlagen ist, werden hier nicht mitgezählt, 
    # sondern nur am Ende bei den nicht empfangenen Nachrichten
    print(missingMessages.sum(), 'messages missing')

    # Diese Anzahl an Nachrichten wird dann erzeugt mit dem Attribut received auf False
    insertedMessages = pd.DataFrame(data=[], columns=['time', 'received'])
    for index, val in missingMessages.iloc[::-1].items():
        time = data['time'][index - 1]
        for x in range(val):
            time = time + pd.Timedelta(seconds=secondsPerMessage)
            insertedMessages = insertedMessages.append({
                'time': time,
                'received': False
            }, ignore_index=True)

    # Neue Einträge hinzufügen und nach Uhrzeit sortieren
    newData = pd.concat([data, insertedMessages], ignore_index=True)
    newData = newData.sort_values('time', ignore_index=True)
    
    
    # Anzahl an gesendeten und empfangen Nachrichten, sowie die Erfolgsrate berechnen
    sendCount = len(newData.index)
    successCount = (newData['received'] == True).sum()
    successRate = successCount / sendCount
    
    print('Sent: ', sendCount)
    print('Received: ', successCount)
    print('Rate: ', successRate)
    

In [3]:
# Zeitraum: 2021-12-27T15:46:05Z – 2022-01-31 ca. 15:00
sensors = ['elsys-ersco2-052869', 'elsys-ersco2-05286b', 'elsys-ersco2-048e6a', 'elsys-ersco2-045181','elsys-ersco2-045182', 'elsys-ersco2-045184', 'elsys-ersco2-045185', 'elsys-ersco2-048e67', 'elsys-ersco2-048e68', 'elsys-ersco2-048e69']

for sensor in sensors:
    getData('Donaubad Elsys Ab Ende Dez.csv', 300, sensor)



elsys-ersco2-052869
29 messages missing
Sent:  10071
Received:  10042
Rate:  0.9971204448416244

elsys-ersco2-05286b
79 messages missing
Sent:  10072
Received:  9993
Rate:  0.9921564733915806

elsys-ersco2-048e6a
5093 messages missing
Sent:  10071
Received:  4978
Rate:  0.49429053718597954

elsys-ersco2-045181
34 messages missing
Sent:  10072
Received:  10038
Rate:  0.9966243050039714

elsys-ersco2-045182
41 messages missing
Sent:  10072
Received:  10031
Rate:  0.9959293089753772

elsys-ersco2-045184
36 messages missing
Sent:  10072
Received:  10036
Rate:  0.9964257347100873

elsys-ersco2-045185
45 messages missing
Sent:  10072
Received:  10027
Rate:  0.9955321683876092

elsys-ersco2-048e67
5099 messages missing
Sent:  10071
Received:  4972
Rate:  0.4936947671532122

elsys-ersco2-048e68
5096 messages missing
Sent:  10071
Received:  4975
Rate:  0.49399265216959587

elsys-ersco2-048e69
5056 messages missing
Sent:  10071
Received:  5015
Rate:  0.4979644523880449


In [4]:
sensors = ['comtac-lpn-di-297329-1111', 'comtac-lpn-di-297330-1118', 'comtac-lpn-di-297331-1119']

for sensor in sensors:
    getData('Donaubad Comtac Ab Ende Dez.csv', 3600, sensor)


comtac-lpn-di-297329-1111
26 messages missing
Sent:  839
Received:  813
Rate:  0.9690107270560191

comtac-lpn-di-297330-1118
32 messages missing
Sent:  839
Received:  807
Rate:  0.9618593563766389

comtac-lpn-di-297331-1119
4 messages missing
Sent:  839
Received:  835
Rate:  0.9952324195470799
